In [1]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd
import numpy as np
from IPython.display import HTML

In [8]:
conn = psycopg2.connect(
    host = "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com",
    port = 5432,
    user = "contratosbasegov",
    password = "F6PL9K4sa^KnH6b^^%BX^Pq!oN93DAnP")

cur = conn.cursor()

OperationalError: connection to server at "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com" (52.58.120.91), port 5432 failed: FATAL:  password authentication failed for user "contratosbasegov"
connection to server at "contratos-base-gov1.cf87yxnqgph8.eu-central-1.rds.amazonaws.com" (52.58.120.91), port 5432 failed: FATAL:  no pg_hba.conf entry for host "89.214.167.25", user "contratosbasegov", database "contratosbasegov", no encryption
